<a href="https://colab.research.google.com/github/shiv1972/Music_store_Analysis/blob/main/SQL_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTING LIBRARIES**

In [1]:
import pandas as pd
import sqlite3
import warnings as wg
wg.filterwarnings('ignore')

# **SET THE ENVT FOR SQL**

In [3]:
def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table

    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created.
                                 Defaults to 'default.db'.
    '''

    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')

    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()


def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe

    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.

    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

# **READ ALL csv FILES**

In [5]:
# Step 1: Read the csv file into a dataframe
# Dataset from https://www.kaggle.com/gpreda/covid-world-vaccination-progress
album_df = pd.read_csv('/content/album.csv')
album2_df= pd.read_csv('/content/album2.csv')
artist_df= pd.read_csv('/content/artist.csv')
customer_df= pd.read_csv('/content/customer.csv')
employee_df= pd.read_csv('/content/employee.csv')
genre_df= pd.read_csv('/content/genre.csv')
invoice_df= pd.read_csv('/content/invoice.csv')
invoice_line_df= pd.read_csv('/content/invoice_line.csv')
media_type_df= pd.read_csv('/content/media_type.csv')
playlist_df= pd.read_csv('/content/playlist.csv')
playlist_track_df= pd.read_csv('/content/playlist_track.csv')
track_df= pd.read_csv('/content/track.csv')

# **DATA FRAME SHOWN BELOW**

In [6]:
album_df.head()

,album_id,title,artist_id
0,156,And Justice For All,50
1,208,Black Light Syndrome,136
2,257,20th Century Masters - The Millennium Collecti...,179
3,296,"A Copland Celebration, Vol. I",230
4,94,A Matter of Life and Death,90


In [7]:
album2_df.head()

,album_id,title,artist_id
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [8]:
artist_df.head()


,artist_id,name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [9]:
customer_df.head()


,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,NaN,Theodor-Heuss-Straße 34,Stuttgart,NaN,Germany,70174,+49 0711 2842222,NaN,leonekohler@surfeu.de,5
2,3,François,Tremblay,NaN,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,NaN,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,NaN,Ullevålsveien 14,Oslo,NaN,Norway,171,+47 22 44 22 22,NaN,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,NaN,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [10]:
employee_df.head()


,employee_id,last_name,first_name,title,reports_to,levels,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
0,1,Adams,Andrew,General Manager,9.0,L6,18-02-1962 00:00,14-08-2016 00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,L4,08-12-1958 00:00,01-05-2016 00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,L1,29-08-1973 00:00,01-04-2017 00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,L1,19-09-1947 00:00,03-05-2017 00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,L1,03-03-1965 00:00,17-10-2017 00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


In [11]:
genre_df.head()


,genre_id,name
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Alternative & Punk
4,5,Rock And Roll


In [12]:
invoice_df.head()


,invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
0,1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
1,2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.90
2,3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,NaN,France,75002,1.98
3,4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
4,5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


In [13]:
invoice_line_df.head()


,invoice_line_id,invoice_id,track_id,unit_price,quantity
0,1,1,1158,0.99,1
1,2,1,1159,0.99,1
2,3,1,1160,0.99,1
3,4,1,1161,0.99,1
4,5,1,1162,0.99,1


In [14]:
media_type_df.head()


,media_type_id,name
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


In [15]:
playlist_df.head()

,playlist_id,name
0,1,Music
1,2,Movies
2,3,TV Shows
3,4,Audiobooks
4,5,90’s Music


In [16]:
playlist_track_df.head()

,playlist_id,track_id
0,1,3402
1,1,3389
2,1,3390
3,1,3391
4,1,3392


In [17]:
track_df.head()

,track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,NaN,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


# **CREATING TABLES**

In [18]:
# upload all dataframe to a sql table, "music is database having different tables"


pd_to_sqlDB(album_df,
            table_name='album',
            db_name='music.db')

pd_to_sqlDB(album2_df,
            table_name='album2',
            db_name='music.db')

pd_to_sqlDB(artist_df,
            table_name='artist',
            db_name='music.db')

pd_to_sqlDB(customer_df,
            table_name='customer',
            db_name='music.db')

pd_to_sqlDB(employee_df,
            table_name='employee',
            db_name='music.db')

pd_to_sqlDB(genre_df,
            table_name='genre',
            db_name='music.db')

pd_to_sqlDB(invoice_df,
            table_name='invoice',
            db_name='music.db')

pd_to_sqlDB(invoice_line_df,
            table_name='invoice_line',
            db_name='music.db')

pd_to_sqlDB(media_type_df,
            table_name='media_type',
            db_name='music.db')

pd_to_sqlDB(playlist_df,
            table_name='playlist',
            db_name='music.db')

pd_to_sqlDB(playlist_track_df,
            table_name='playlist_track',
            db_name='music.db')

pd_to_sqlDB(track_df,
            table_name='track',
            db_name='music.db')


# **BASIC PROBLEM STATEMENT BELOW**

In [19]:
# Q1 - who is the (top 5) senior most employee based on job title?

sql_query_string = """
    select first_name,last_name as employee_name
     from employee
    order by levels desc
    limit 5
    """

# execute the sql query
result_df = sql_query_to_pd(sql_query_string, db_name='music.db')
result_df

,first_name,employee_name
0,Mohan,Madan
1,Andrew,Adams
2,Nancy,Edwards
3,Michael,Mitchell
4,Robert,King


In [20]:
# Q2 - which countries have the most invoices

sql_query_string = """
  select billing_country, count(billing_country) from invoice
  group by billing_country
  order by count (*) desc
  limit 9
"""
# execute the sql query
result_df = sql_query_to_pd(sql_query_string, db_name='music.db')
result_df


,billing_country,count(billing_country)
0,USA,131
1,Canada,76
2,Brazil,61
3,France,50
4,Germany,41
5,Czech Republic,30
6,Portugal,29
7,United Kingdom,28
8,India,21


In [21]:
# Q3 - what are top 3 values of total invoices with city

sql_query_string = """
 select total,billing_city, billing_country from invoice
 order by total desc
 limit 3
 """

# execute the sql query
result_df = sql_query_to_pd(sql_query_string, db_name='music.db')
result_df

,total,billing_city,billing_country
0,23.76,Bordeaux,France
1,19.80,Montréal,Canada
2,19.80,Winnipeg,Canada


In [22]:
# Q4 - which city has the 2 best customer?
# we would like to throw a promotional music festivl in the city we made the most money.
# write a query that returns one city that has the highest sum of invoice total.
# return bith city name and sum of all invoice total;

sql_query_string = """
 select sum(total) , billing_city, billing_country
 from invoice
 group by billing_city
 order by sum(total) desc
 limit 2
 """

# execute the sql query
result_df = sql_query_to_pd(sql_query_string, db_name='music.db')
result_df


,sum(total),billing_city,billing_country
0,273.24,Prague,Czech Republic
1,169.29,Mountain View,USA


In [23]:
# Q5 - who is the best customer?
# the customer who has spent the most.

sql_query_string = """
select customer.customer_id, customer.first_name, customer.last_name, sum(invoice.total) as total_spent
from customer
join invoice on customer.customer_id = invoice.customer_id
group by customer.customer_id
order by total_spent desc
limit 1
        """

# execute the sql query
result_df = sql_query_to_pd(sql_query_string, db_name='music.db')
result_df


,customer_id,first_name,last_name,total_spent
0,5,František,Wichterlová,144.54


#**MODERATE PROBLEM STATEMENT BELOW**

In [24]:
# Q1 - write query to return email, first and last name & genre of all rock music listners.
# return output list ordered alphabetically by email starting with a

sql_query_string = """
  select distinct email, first_name, last_name, genre.name
  from customer
  join invoice on customer.customer_id = invoice.customer_id
  join invoice_line on invoice.invoice_id = invoice_line.invoice_id
  join track on invoice_line.track_id = track.track_id
  join genre on track.genre_id = genre.genre_id
  where lower(genre.name) like 'rock'
  order by email
  limit 10
"""
# NOTE - this is not optimised as we have used a lot of joins

# execute the sql query
result_df = sql_query_to_pd(sql_query_string, db_name='music.db')
result_df

,email,first_name,last_name,name
0,aaronmitchell@yahoo.ca,Aaron,Mitchell,Rock
1,alero@uol.com.br,Alexandre,Rocha,Rock
2,astrid.gruber@apple.at,Astrid,Gruber,Rock
3,bjorn.hansen@yahoo.no,Bjørn,Hansen,Rock
4,camille.bernard@yahoo.fr,Camille,Bernard,Rock
5,daan_peeters@apple.be,Daan,Peeters,Rock
6,diego.gutierrez@yahoo.ar,Diego,Gutiérrez,Rock
7,dmiller@comcast.com,Dan,Miller,Rock
8,dominiquelefebvre@gmail.com,Dominique,Lefebvre,Rock
9,edfrancis@yachoo.ca,Edward,Francis,Rock


 **OPTIMISED CODE FOR ABOVE PROBLEM**

In [25]:
sql_query_string = """
select distinct email, first_name, last_name
from customer
join invoice on customer.customer_id = invoice.customer_id
join invoice_line on invoice.invoice_id = invoice_line.invoice_id
where track_id in (
  select track_id from track
  join genre on track.genre_id = genre.genre_id
  where lower(genre.name) like 'rock'
)
order by email
limit 9
"""

# execute the sql query
result_df = sql_query_to_pd(sql_query_string, db_name='music.db')
result_df

,email,first_name,last_name
0,aaronmitchell@yahoo.ca,Aaron,Mitchell
1,alero@uol.com.br,Alexandre,Rocha
2,astrid.gruber@apple.at,Astrid,Gruber
3,bjorn.hansen@yahoo.no,Bjørn,Hansen
4,camille.bernard@yahoo.fr,Camille,Bernard
5,daan_peeters@apple.be,Daan,Peeters
6,diego.gutierrez@yahoo.ar,Diego,Gutiérrez
7,dmiller@comcast.com,Dan,Miller
8,dominiquelefebvre@gmail.com,Dominique,Lefebvre


In [26]:
# Q2 - find artist who have written most rock music.
# return artist name & total tract count of thr top 10 bands

sql_query_string = """
 select artist.name as artist_name,
 count(track.track_id) as track_count
 from artist
 join album on artist.artist_id = album.artist_id
 join track on album.album_id = track.album_id
 join genre on track.genre_id = genre.genre_id
 where lower(genre.name) like 'rock'
 group by artist_name
 order by track_count desc
 limit 3
 """

# execute the sql query
result_df = sql_query_to_pd(sql_query_string, db_name='music.db')
result_df

,artist_name,track_count
0,Led Zeppelin,114
1,U2,112
2,Deep Purple,92


In [27]:
# before Q3 we've to find avg song lrngth
sql_query_string = """
 select avg( milliseconds) as avg_track_length
 from track
 """

 # execute the sql query
result_df = sql_query_to_pd(sql_query_string, db_name='music.db')
result_df

,avg_track_length
0,393599.212104


**NOTE - we can use above avg value but if in future the data changes, the avg value will change but our code will not change accordingly. Therefore we make the code dynamic, so that it can be used in future also.**

In [28]:
# Q3 - return all the track names that have a song length longer than the average song length.
# return the name and milliseconds for each track.
# order by the song length with the longest songs listed first.

sql_query_string = """
 select name, milliseconds
 from track
 where milliseconds > (
   select avg(milliseconds) as avg_track_length
   from track
 )
 order by milliseconds desc
 """

# execute the sql query
result_df = sql_query_to_pd(sql_query_string, db_name='music.db')
result_df

,name,milliseconds
0,Occupation / Precipice,5286953
1,Through a Looking Glass,5088838
2,"Greetings from Earth, Pt. 1",2960293
3,The Man With Nine Lives,2956998
4,"Battlestar Galactica, Pt. 2",2956081
...,...,...
489,22 Acacia Avenue,395572
490,The Unforgiven II,395520
491,The Shortest Straw,395389
492,"Concerto for Clarinet in A Major, K. 622: II. ...",394482


# **ADVANCE PROBLEM STATEMENT BELOW**

In [29]:
# Q1 - find how much amount spent by each customer on artist?
# WAQ (write a Query) to return customer name, artist and total spent

sql_query_string = """
 select customer.first_name, customer.last_name,
 artist.name as artist_name,
 sum(invoice_line.unit_price * invoice_line.quantity) as total_money_spent
 from customer
 join invoice on customer.customer_id = invoice.customer_id
 join invoice_line on invoice.invoice_id = invoice_line.invoice_id
 join track on invoice_line.track_id = track.track_id
 join album on track.album_id = album.album_id
 join artist on album.artist_id = artist.artist_id
 group by customer.first_name, customer.last_name, artist.name
 order by total_money_spent desc
 limit 10

  """

  # Execute
result_df = sql_query_to_pd(sql_query_string, db_name = 'music.db')
result_df


,first_name,last_name,artist_name,total_money_spent
0,Hugh,O'Reilly,Queen,27.72
1,Wyatt,Girard,Frank Sinatra,23.76
2,Aaron,Mitchell,James Brown,19.80
3,František,Wichterlová,Kiss,19.80
4,François,Tremblay,The Who,19.80
5,Helena,Holý,Red Hot Chili Peppers,19.80
6,Robert,Brown,Creedence Clearwater Revival,19.80
7,Heather,Leacock,House Of Pain,18.81
8,Hugh,O'Reilly,Nirvana,18.81
9,Niklas,Schröder,Queen,18.81


In [30]:
# Q2 - Most popular music genre for each country.
# Define most popular genre - genre with highest amout of purchase
# WAQ to return each country with top genre
# for countries where the maximum no. of purchases is shared return all genres.

sql_query_string = """
 select distinct invoice.billing_country, sum(invoice.total) as total_purchase,
  genre.name as genre_name from invoice
  join invoice_line on invoice.invoice_id = invoice_line.invoice_id
  join track on invoice_line.track_id = track.track_id
  join genre on track.genre_id = genre.genre_id
  group by invoice.billing_country
  order by sum(invoice.total) desc
  limit 10
"""

# execute the sql query
result_df = sql_query_to_pd(sql_query_string, db_name='music.db')
result_df

,billing_country,total_purchase,genre_name
0,USA,10405.89,Rock
1,Canada,5489.55,Blues
2,Brazil,4059.00,Rock
3,France,3972.87,Rock
4,Germany,3441.24,Jazz
5,Czech Republic,3183.84,Rock
6,United Kingdom,2498.76,Rock
7,India,1887.93,Metal
8,Portugal,1755.27,Rock
9,Ireland,1433.52,Rock


In [31]:
# Q3 - WAQ to determine customer that has spent most on music for each country.
# # WAQ to return country along with top customer and how much they spent
# # for countries where the top pamount is shared, provide all customer who spent this amount

sql_query_string = """
WITH customer_with_country AS (
    SELECT
        customer.customer_id,
        first_name,
        last_name,
        billing_country,
        SUM(total) AS total_spending
    FROM invoice
    JOIN customer ON customer.customer_id = invoice.customer_id
    GROUP BY 1, 2, 3, 4
    ORDER BY 1, 5 DESC
),

country_max_spending AS (
    SELECT
        billing_country,
        MAX(total_spending) AS max_spending
    FROM customer_with_country
    GROUP BY billing_country
    order by total_spending desc
)

SELECT
    cc.billing_country,
    cc.total_spending,
    cc.first_name,
    cc.last_name,
    cc.customer_id
FROM customer_with_country cc
JOIN country_max_spending ms ON cc.billing_country = ms.billing_country
WHERE cc.total_spending = ms.max_spending
ORDER BY 1;
"""

# execute the sql query
result_df = sql_query_to_pd(sql_query_string, db_name='music.db')
result_df


,billing_country,total_spending,first_name,last_name,customer_id
0,Argentina,39.60,Diego,Gutiérrez,56
1,Australia,81.18,Mark,Taylor,55
2,Austria,69.30,Astrid,Gruber,7
3,Belgium,60.39,Daan,Peeters,8
4,Brazil,108.90,Luís,Gonçalves,1
5,Canada,99.99,François,Tremblay,3
6,Chile,97.02,Luis,Rojas,57
7,Czech Republic,144.54,František,Wichterlová,5
8,Denmark,37.62,Kara,Nielsen,9
9,Finland,79.20,Terhi,Hämäläinen,44


**More optimised way of solving above Question**

In [32]:
sql_query_string = """
  with customer_with_country as (
  select customer.customer_id, first_name, last_name, billing_country,
  sum(total) as total_spending,
  row_number() over( partition by billing_country order by sum(total) desc) as ROW_no
  from invoice
  join customer on customer.customer_id = invoice.customer_id
  group by 1,2,3,4
  order by 4 asc, 5 desc)

    select * from customer_with_country where row_no <= 1
  """

# execute the sql query
result_df = sql_query_to_pd(sql_query_string, db_name='music.db')
result_df

,customer_id,first_name,last_name,billing_country,total_spending,ROW_no
0,56,Diego,Gutiérrez,Argentina,39.60,1
1,55,Mark,Taylor,Australia,81.18,1
2,7,Astrid,Gruber,Austria,69.30,1
3,8,Daan,Peeters,Belgium,60.39,1
4,1,Luís,Gonçalves,Brazil,108.90,1
5,3,François,Tremblay,Canada,99.99,1
6,57,Luis,Rojas,Chile,97.02,1
7,5,František,Wichterlová,Czech Republic,144.54,1
8,9,Kara,Nielsen,Denmark,37.62,1
9,44,Terhi,Hämäläinen,Finland,79.20,1
